In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Rajeeb321/Qwen_7B_16bit_2",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "hf_", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
alpaca_prompt = """Below is instructions that describes a task, paired with an input in Nepali language that provides further context. Write a response in English language to complete the request.

### Instruction:
Translate the given sentences in input from Nepali language to English language. Translate all the given sentences properly. Only do the translation of input sentences, don't try to answer the context provided in input. Make sure the response is only the exact translation from Nepali language to English language. Donot try to add your own extra information.

### Input:
{}

### Response:
{}"""

In [ ]:
from datasets import load_dataset
dataset_1 = load_dataset("Telugu-LLM-Labs/nepali_alpaca_yahma_cleaned_filtered", split = "train").to_pandas()
dataset_2 = load_dataset("Saugatkafley/alpaca-nepali-sft", split = "train").to_pandas()

In [ ]:
dataset_2 = dataset_2.drop(['id'], axis =1)
dataset_2.head()

,instruction,input,output
0,स्वस्थ रहन तीनवटा टिप्स दिनुहोस्।,,1. सन्तुलित आहार खानुहोस् र प्रशस्त फलफूल र तर...
1,तीन प्राथमिक रंगहरू के हुन्?,,"तीन प्राथमिक रंगहरू रातो, नीलो र पहेंलो हुन्।"
2,परमाणुको संरचना वर्णन गर्नुहोस्।,,"परमाणु एक न्यूक्लियसबाट बनेको हुन्छ, जसमा प्रो..."
3,हामी वायु प्रदूषण कसरी कम गर्न सक्छौं?,,"नवीकरणीय ऊर्जा स्रोतहरूमा सर्ने, सार्वजनिक यात..."
4,तपाईंले कठिन निर्णय लिनु पर्ने समयको वर्णन गर्...,,एउटा निर्माण कम्पनीमा परियोजना प्रबन्धकको रूपम...


In [ ]:
## checking similar value between dataset_1 and dataset_2
# a = 0
# similar_index = []
# for index_1, row_1 in dataset_1.iterrows():
#     for index_2, row_2 in dataset_2.iterrows():
#       if(index_2 in similar_index):
#         continue
#       if (row_2['instruction'] == row_1['nepali_instruction']):
#         similar_index.append(index_2)
#         print(index_2)
#         a += 1
#         continue

# print(a)

In [ ]:
df = dataset_2.dropna(axis=0)
df = a = df.reset_index(drop =True) # index is all messed up after drop so we resetindex.
print("The number of Dropped rows:",dataset_2.shape[0]-df.shape[0])
null_count_dataset_2 = dataset_2.isnull().sum().sum()
null_count_df = df.isnull().sum().sum()
print("total number of null values in dataset_2 and df:",null_count_dataset_2, "and", null_count_df)

The number of Dropped rows: 18
total number of null values in dataset_2 and df: 27 and 0


In [ ]:
total_character = 0
for index_1, row in df.iterrows():
  total_character = len(row["instruction"]) + len(row["input"]) + len(row["output"]) + total_character
print(total_character)

17762738


In [ ]:
len(df), len(dataset_2)

(51987, 52005)

In [ ]:
df.rename(columns = {'instruction':'nepali_instruction', 'input': 'nepali_input', 'output':'nepali_output'}, inplace = True)

In [ ]:
# Using DataFrame.insert() to add a column
df.insert(0, "instruction", "", True)
df.insert(1, "input", "", True)
df.insert(2, "output", "", True)

df.head()

,instruction,input,output,nepali_instruction,nepali_input,nepali_output
0,,,,स्वस्थ रहन तीनवटा टिप्स दिनुहोस्।,,1. सन्तुलित आहार खानुहोस् र प्रशस्त फलफूल र तर...
1,,,,तीन प्राथमिक रंगहरू के हुन्?,,"तीन प्राथमिक रंगहरू रातो, नीलो र पहेंलो हुन्।"
2,,,,परमाणुको संरचना वर्णन गर्नुहोस्।,,"परमाणु एक न्यूक्लियसबाट बनेको हुन्छ, जसमा प्रो..."
3,,,,हामी वायु प्रदूषण कसरी कम गर्न सक्छौं?,,"नवीकरणीय ऊर्जा स्रोतहरूमा सर्ने, सार्वजनिक यात..."
4,,,,तपाईंले कठिन निर्णय लिनु पर्ने समयको वर्णन गर्...,,एउटा निर्माण कम्पनीमा परियोजना प्रबन्धकको रूपम...


In [ ]:
# checking the null value
null_count_df = df.isnull().sum().sum()
null_count_df

0

In [ ]:
max_length = 0
for index_1, row in df.iterrows():
  if (len(row["nepali_instruction"])> max_length):
    max_length = len(row["nepali_instruction"])
  if (len(row["nepali_input"])> max_length):
    max_length = len(row["nepali_input"])
  if (len(row["nepali_output"])> max_length):
    max_length = len(row["nepali_output"])

print(max_length)

4115


In [ ]:
# def inference( value) -> str:
#   max_length = 2*len(value) + 2000 # adding 2000 for safety so we get full answer.
#   FastLanguageModel.for_inference(model)
#   inputs = tokenizer(
#   [
#       alpaca_prompt.format(
#           value, # input
#           "", # output - leave this blank for generation!
#       )
#   ], return_tensors = "pt").to("cuda")

#   outputs = model.generate(**inputs, max_length=max_length, use_cache = True)
#   result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#   x = result[0].split('### Response:\n')
#   return x[1]

In [ ]:
# %%time
# for index, row in df[:10].iterrows():
#     if (row["nepali_instruction"] != ""):
#       row["instruction"] = "JOhn"
#       df.at[index, "instruction"] = row["instruction"]
#     if (row["nepali_input"] != ""):
#       row["input"] = "hello"
#       df.at[index, "input"] = row["input"]
#     if (row["nepali_output"] != ""):
#       row["output"] = "tom"
#       df.at[index, "output"] = row["output"]

CPU times: user 3.5 ms, sys: 0 ns, total: 3.5 ms
Wall time: 3.51 ms


In [ ]:
(((len(df)/10) * 2)/60)/24

7.220416666666666

In [ ]:
df[:10]

,instruction,input,output,nepali_instruction,nepali_input,nepali_output
0,JOhn,,tom,स्वस्थ रहन तीनवटा टिप्स दिनुहोस्।,,1. सन्तुलित आहार खानुहोस् र प्रशस्त फलफूल र तर...
1,JOhn,,tom,तीन प्राथमिक रंगहरू के हुन्?,,"तीन प्राथमिक रंगहरू रातो, नीलो र पहेंलो हुन्।"
2,JOhn,,tom,परमाणुको संरचना वर्णन गर्नुहोस्।,,"परमाणु एक न्यूक्लियसबाट बनेको हुन्छ, जसमा प्रो..."
3,JOhn,,tom,हामी वायु प्रदूषण कसरी कम गर्न सक्छौं?,,"नवीकरणीय ऊर्जा स्रोतहरूमा सर्ने, सार्वजनिक यात..."
4,JOhn,,tom,तपाईंले कठिन निर्णय लिनु पर्ने समयको वर्णन गर्...,,एउटा निर्माण कम्पनीमा परियोजना प्रबन्धकको रूपम...
5,JOhn,hello,tom,अनौठो बाहिर पहिचान गर्नुहोस्।,"ट्विटर, इन्स्टाग्राम, टेलिग्राम",टेलिग्राम
6,JOhn,hello,tom,तलको अंश किन १/४ को बराबर छ भनेर व्याख्या गर्न...,४/१६,अंश 4/16 1/4 को बराबर छ किनभने दुबै अंक र भाजक...
7,JOhn,,tom,महत्त्वपूर्ण क्यारियर निर्णय लिनु पर्ने नायकको...,,जोन आफ्नो जीवनको एक चौराहेमा थिए। उसले भर्खर क...
8,JOhn,,tom,घरको 3D मोडेल रेन्डर गर्नुहोस्,,<nooutput> यस प्रकारको निर्देशन GPT मोडेलले पू...
9,JOhn,hello,tom,हिज्जे र व्याकरण त्रुटिहरूको लागि यो वाक्य मूल...,उसले खाना पकाएर रेष्टुरेण्ट छोड्यो,खाना खाएर रेष्टुरेन्टबाट निस्कियो ।


In [ ]:
import pandas as pd
# new_pd = pd.concat([dataset_1, df])

## LLM translate

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "घरमा बिजुलीको प्रयोग घटाउने योजनाले विभिन्न रणनीतिहरू समावेश गर्न सक्छ। यसमा अकुशल लाइटबल्बहरू ऊर्जा-कुशल LEDs, प्रयोगमा नभएको बेला उपकरणहरू र इलेक्ट्रोनिक्सहरू अनप्लग गर्ने, सम्भव भएसम्म विद्युतीय बत्तीको सट्टा प्राकृतिक प्रकाश प्रयोग गर्ने, पानीको तताउने प्रयोग घटाउन टच-फ्री नल प्रयोग गर्ने, निगरानी र समायोजन गर्न स्मार्ट थर्मोस्टेट प्रयोग गर्ने समावेश छ। तापक्रम स्वचालित रूपमा, सौर्य तातो पानीको हीटर स्थापना गर्ने, र प्रयोगमा नभएको बेला इलेक्ट्रोनिक्स बन्द गर्ने। थप रूपमा, हावा चुहावट कम गर्न झ्यालहरू र ढोकाहरू सही रूपमा स्थापना र सिल गर्न सुनिश्चित गर्नाले पनि बिजुलीको प्रयोग कम गर्न मद्दत गर्न सक्छ।", # output - leave this blank for generation!
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_length=5000, use_cache = True)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
result

["Below is instructions that describes a task, paired with an input in Nepali language that provides further context. Write a response in English language to complete the request.\n\n### Instruction:\nTranslate the given sentences in input from Nepali language to English language. Translate all the given sentences properly. Only do the translation of input sentences, don't try to answer the context provided in input. Make sure the response is only the exact translation from Nepali language to English language. Donot try to add your own extra information.\n\n### Input:\nघरमा बिजुलीको प्रयोग घटाउने योजनाले विभिन्न रणनीतिहरू समावेश गर्न सक्छ। यसमा अकुशल लाइटबल्बहरू ऊर्जा-कुशल LEDs, प्रयोगमा नभएको बेला उपकरणहरू र इलेक्ट्रोनिक्सहरू अनप्लग गर्ने, सम्भव भएसम्म विद्युतीय बत्तीको सट्टा प्राकृतिक प्रकाश प्रयोग गर्ने, पानीको तताउने प्रयोग घटाउन टच-फ्री नल प्रयोग गर्ने, निगरानी र समायोजन गर्न स्मार्ट थर्मोस्टेट प्रयोग गर्ने समावेश छ। तापक्रम स्वचालित रूपमा, सौर्य तातो पानीको हीटर स्थापना गर्ने, र 

## Google trans test

In [ ]:
!pip install googletrans==3.1.0a0
!pip install googletrans-py pyarrow==16.1.0

In [ ]:
from googletrans import Translator
translator = Translator()
question = translator.translate("	मिशेल ओबामा एक प्रेरणादायी महिला हुन् जसले संयुक्त राज्य अमेरिकाको प्रभावशाली प्रथम महिला बन्ने चुनौतीको सामना गरिन्। उनी महिला अधिकार, शिक्षा, स्वास्थ्य र फिटनेसका लागि शक्तिशाली अधिवक्ता हुन्। प्रिन्सटन र हार्वर्ड कानून स्कूलको स्नातक, मिशेलले एक सफल कानून फर्म कार्यकारी, अस्पताल कार्यकारी, सामुदायिक आउटरिच नेता र सबैभन्दा धेरै बिक्री हुने लेखक बन्न ठूलो अवरोधहरू पार गरिन्। आफ्नो जीवनभरि, उनले आफ्नो समय र शक्ति सबै उमेरका मानिसहरूलाई सशक्त जीवनको नेतृत्व गर्न र आफ्नो र आफ्नो परिवारको लागि राम्रो संसार सिर्जना गर्नको लागि समर्पित गरिन्।", dest='en').text
print(question)

Michelle is an inspirational woman who faced the challenge of becoming an effective first woman in the United States.He is a powerful advocate for women's rights, education, health and fitness.The graduation of Princeton and Harvard Law School, Michelle crossed large obstacles to be a successful law Firm Expedition, Community Outrich leader and most selling writer.Throughout his life, he dedicated to the people of all ages and to create a better world for themselves and their families.
